In [1]:
import os
import re
import sys
import pandas as pd
import glob
import numpy as np

import itertools
from itertools import tee


In [2]:
virus_coord_dir = "/Users/u241374/mike_tisza/sandbox/test_ct2_0802e/ct2_tmp/assess_prune/indiv_seqs"

gene_annotation_file = "/Users/u241374/mike_tisza/sandbox/test_ct2_0802e/ct2_tmp/assess_prune/contig_gene_annotation_summary.tsv"

oriented_contigs_file = "/Users/u241374/mike_tisza/sandbox/test_ct2_0802e/ct2_tmp/oriented_hallmark_contigs.fasta"

In [3]:
try:

    coord_files = glob.glob(os.path.join(virus_coord_dir, "*.viruses.tsv"))

    df_from_each_coord = (pd.read_csv(coord, sep = "\t", header = None,
                                    names = ["contig", "chunk_start", "chunk_stop", 
                                             "chunk_name", "hallmark_count"])
                                    for coord in coord_files)
    chunk_coord_df = pd.concat(df_from_each_coord, ignore_index=True)

    #chunk_coord_df.head()

except:
    print("no chunk coords")

chunk_coord_df.head()

,contig,chunk_start,chunk_stop,chunk_name,hallmark_count
0,test_ct2_0802e3952,0,12965,Chunk_0,6
1,test_ct2_0802e4843,0,11550,Chunk_0,11
2,test_ct2_0802e4767,0,19541,Chunk_0,4
3,test_ct2_0802e43597,0,62984,Chunk_0,5
4,test_ct2_0802e4099,0,17549,Chunk_0,10


In [5]:
adjusted_gene_df = pd.DataFrame()

gene_contig_df = pd.read_csv(gene_annotation_file, sep = "\t")

grouped_df = gene_contig_df.groupby('contig')

for name, group in grouped_df:
    contig_length1 = group['contig_length'].agg(pd.Series.mode)

    dtr_status = group['dtr_seq'].agg(pd.Series.mode)

    if len(dtr_status) == 0 and int(contig_length1.iloc[0]) >=10000:
        ## lift genes from each chunk

        ## @name is how to call the name variable in query
        thisgroup_chunk_df = chunk_coord_df.query("contig == @name")

        for index, row in thisgroup_chunk_df.iterrows():
            chunk_start = row.loc['chunk_start'] - 1
            if chunk_start < 0:
                chunk_start = 0
            chunk_stop = row.loc['chunk_stop']
            chunk_name = row.loc['chunk_name']
            
            thischunk_gene_df = group.copy().query("gene_start >= @chunk_start & gene_stop <= @chunk_stop")

            thischunk_gene_df['gene_start'] = thischunk_gene_df['gene_start'] - chunk_start
            thischunk_gene_df['gene_stop'] = thischunk_gene_df['gene_stop'] - chunk_start
            thischunk_gene_df['chunk_name'] = chunk_name
            thischunk_gene_df['chunk_length'] = chunk_stop - chunk_start
            thischunk_gene_df['chunk_start'] = chunk_start
            thischunk_gene_df['chunk_stop'] = chunk_stop

            adjusted_gene_df = pd.concat([adjusted_gene_df, thischunk_gene_df], ignore_index=True)

    else:
        #dtrs and seqs under 10kb
        ## for these just return the gene coordinates as is
        ## make chunk column

        group['chunk_name'] = "NA"
        group['chunk_length'] = "NA"
        group['chunk_start'] = 0
        group['chunk_stop'] = group['contig_length']

        adjusted_gene_df = pd.concat([adjusted_gene_df, group], ignore_index=True)



adjusted_gene_df.head()

#adjusted_gene_df.to_csv("")


,contig,gene_start,gene_stop,gene_name,gene_orient,contig_length,dtr_seq,evidence_acession,evidence_description,Evidence_source,vscore_category,chunk_name,chunk_length,chunk_start,chunk_stop
0,test_ct2_0802e103,1,379,test_ct2_0802e103_9,+,6095,NaN,CDD:COG4653,major-capsid-protein,hallmark_hmm,common_virus,NA,NA,0,6095
1,test_ct2_0802e103,432,999,test_ct2_0802e103_10,+,6095,NaN,CDD:COG3740,head-maturation-protease,hallmark_hmm,common_virus,NA,NA,0,6095
2,test_ct2_0802e103,1000,2233,test_ct2_0802e103_11,+,6095,NaN,CDD:COG4695,portal-protein,hallmark_hmm,common_virus,NA,NA,0,6095
3,test_ct2_0802e103,2216,2573,test_ct2_0802e103_12,+,6095,NaN,PDB:2KZ4_A,Putative-head-tail-adaptor,hallmark_hmm,common_virus,NA,NA,0,6095
4,test_ct2_0802e103,2559,2874,test_ct2_0802e103_13,+,6095,NaN,CDD:TIGR01560,uncharacterized-phage-protein-(possible-DNA-pa...,hallmark_hmm,common_virus,NA,NA,0,6095


In [9]:
bed_df = adjusted_gene_df[['contig', 'chunk_start', 'chunk_stop', 'chunk_name']].drop_duplicates()

bed_df['new_chunk'] = np.where(bed_df['chunk_name'] == "NA", bed_df['contig'], bed_df['contig'] + "@" + bed_df['chunk_name'])

bed_df = bed_df.drop('chunk_name', axis = 1)

bed_df.head()
#bed_df.to_csv("/Users/u241374/mike_tisza/sandbox/caccae_chunks.bed", sep = "\t", 
#              index = False, header = False)

,contig,chunk_start,chunk_stop,new_chunk
0,test_ct2_0802e103,0,6095,test_ct2_0802e103
10,test_ct2_0802e1136,0,3456,test_ct2_0802e1136
14,test_ct2_0802e1184,0,1103,test_ct2_0802e1184
16,test_ct2_0802e1537,0,1024,test_ct2_0802e1537
18,test_ct2_0802e1610,0,1820,test_ct2_0802e1610
